In [1]:
!python -m pip install --user --upgrade pip

     |████████████████████████████████| 1.5MB 18.6MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 18.3 MB/s 
     |████████████████████████████████| 636 kB 39.0 MB/s 
     |████████████████████████████████| 1.5 MB 40.7 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 4.7 MB/s 
     |████████████████████████████████| 1.0 MB 50.9 MB/s 
     |████████████████████████████████| 61 kB 6.0 MB/s 
  Created wheel for kfp: filename=kfp-1.4.0-py3-none-any.whl size=221064 sha256=1f5f3388433f0f38361071f019e0dcce75a371834865e631934bc7e06c466751
  Stored in directory: /root/.ca

In [11]:
import kfp
from kfp import dsl
import kfp.components as comp

In [12]:
def load_data_op():
    return dsl.ContainerOp(
        name = 'Load Data',
        image = 'mavencodevv/load_nba:v.0.1',
        arguments = [],
        file_outputs={
            'data': '/load_data/data'
        }      
    )

In [13]:
def preprocess_op(data):
    return dsl.ContainerOp(
        name = 'Preprocess Data',
        image = 'mavencodevv/preprocess_nba:v.0.1',
        arguments = ['--data', data],
        file_outputs={
            'clean_data': '/preprocess/clean_data'   
        }
    )

In [14]:
def rf_op(clean_data):
    return dsl.ContainerOp(
        name = 'Randomforest',
        image = 'mavencodevv/rf_nba:v.0.2',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'rf_metrics': '/randomforest/rf_metrics' 
        }
    )

In [15]:
def gbc_op(clean_data):
    return dsl.ContainerOp(
        name = 'GBC',
        image = 'mavencodevv/gbc_nba:v.0.2',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'gbc_metrics': '/gbc/gbc_metrics' 
        }
    )

In [16]:
def lr_op(clean_data):
    return dsl.ContainerOp(
        name = 'Logistic Regression',
        image = 'mavencodevv/lr_nba:v.0.2',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'lr_metrics': '/lr/lr_metrics' 
        }
    )

In [17]:
def results(rf_metrics,gbc_metrics,lr_metrics):
    return dsl.ContainerOp(
        name = 'Results',
        image = 'mavencodevv/results_nba:v.0.1',
        arguments = ['--rf_metrics', rf_metrics,
                     '--gbc_metrics', gbc_metrics,
                     '--lr_metrics', lr_metrics
                     ] 
        
    )

In [18]:
@dsl.pipeline(
    name='Shot Result Prediction',
   description='An ML reusable pipeline that predicts whether a shot from an NBA player will go in or not'
)

# Define parameters to be fed into pipeline
def nba_pipeline(): 

  _load_data_op = load_data_op()
    
  _preprocess_op = preprocess_op(
        dsl.InputArgumentPath(_load_data_op.outputs['data'])).after(_load_data_op)
    
  _rf_op = rf_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)

  _gbc_op = gbc_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
  _lr_op = lr_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
   
  _results_op = results(
        dsl.InputArgumentPath(_rf_op.outputs['rf_metrics']),
        dsl.InputArgumentPath(_gbc_op.outputs['gbc_metrics']),
        dsl.InputArgumentPath(_lr_op.outputs['lr_metrics'])).after(_rf_op,_gbc_op,_lr_op)
    

In [19]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'nba_pipeline'
#kfp.compiler.Compiler().compile(churn_reuseable_tensorflow_pipeline,  
#  '{}.zip'.format(experiment_name))
kfp.compiler.Compiler().compile(nba_pipeline, '{}.yaml'.format(experiment_name))

/root/.local/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


In [ ]:
#client = kfp.Client()
#client.create_run_from_pipeline_func(nba_pipeline, arguments={})